<a href="https://colab.research.google.com/github/54wxp/Decision-trees/blob/master/%E2%80%9Cunsloth_finetune_deepseek_r1_with_medical_dataset_to_ollama_ipynb%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Unsloth Fine-tuning DeepSeek R1 Distilled Llama 8B

In this notebook, it will demonstrate how to finetune `DeepSeek-R1-Distill-Llama-8B` with Unsloth, using a medical dataset.

## Why do we need LLM fine-tuning?

Fine-tuning tailors the model to have a better performance for specific tasks, making it more effective and versatile in real-world applications. This process is essential for tailoring an existing model to a particular task or domain.

In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip install bitsandbytes unsloth_zoo
!pip install --upgrade transformers peft
!pip show transformers peft unsloth


## Choose a Base Model

1. Choose a model that aligns with your usecase
2. Assess your storage, compute capacity and dataset
3. Select a Model and Parameters
4. Choose Between Base and Instruct Models

In [4]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-Distill-Qwen-1.5B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.15: Fast Qwen2 patching. Transformers: 4.49.0.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.81G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/6.78k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

## Inference before fine-tuning

In [5]:
prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.
Please answer the following medical question.

### Question:
{}

### Response:
<think>{}"""

In [6]:
question = "根据描述，一个1岁的孩子在夏季头皮出现多处小结节，长期不愈合，且现在疮大如梅，溃破流脓，口不收敛，头皮下有空洞，患处皮肤增厚。这种病症在中医中诊断为什么病？"


FastLanguageModel.for_inference(model)
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])


<think>
好的，现在我要分析这位用户的问题。首先，他提到的是一个1岁的孩子，夏天气温，头皮出现多处小结节，长期不愈合，现在症状是溃破流脓，口不收敛，头皮下有空洞，皮肤增厚。然后问这在中医诊断为什么病。

首先，我需要理解这些症状在中医中的诊断依据。中医有多种理论，比如温病、寒病、湿热湿寒理论，以及阴阳五行理论。小结节可能与湿热有关，因为湿热会导致结节。口不收敛可能与火证有关，因为火证会导致口生风，不收敛。皮肤增厚可能与湿热有关，因为湿热容易引起皮肤问题。

接下来，我需要考虑患者是否有发热、干咳等其他症状，因为这些可能影响诊断。患者没有提到这些，所以可能需要根据症状推断。小结节多，长期不愈合，说明可能有湿热湿寒证，因为湿热容易导致结节，长期不愈合可能提示湿热湿寒。

另外，口不收敛可能与湿热湿寒有关，因为湿热会导致口生风，不收敛。头皮下有空洞可能与湿热湿寒有关，因为湿热容易引起头皮发炎，空洞可能是因为湿热扩散。

综合考虑，这可能属于湿热湿寒证，因为小结节多，口不收敛，头皮有空洞，这些都与湿热湿寒相关。因此，中医诊断为湿热湿寒证。
</think>

根据描述，患者的情况属于湿热湿寒证。湿热湿寒证是中医的一种证候，常见于湿热过度或湿寒不足引起的湿热湿寒。湿热湿寒证患者常伴有发热、干咳、咳嗽、多汗等症状，但通常不需要药物治疗。在本例中，患者有小结节多、口不收敛、皮肤增厚等症状，符合湿热湿寒证的特征。因此，中医诊断为湿热湿寒证。<｜end▁of▁sentence｜>


## Prepare Dataset

A medical dataset [https://huggingface.co/datasets/FreedomIntelligence/medical-o1-reasoning-SFT/](https://huggingface.co/datasets/FreedomIntelligence/medical-o1-reasoning-SFT/) will be used to train the selected model.

In [7]:
train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.
Please answer the following medical question.

### Question:
{}

### Response:
<think>
{}
</think>
{}"""

### Important Notice

It's crucial to add the EOS (End of Sequence) token at the end of each training dataset entry, otherwise you may encounter infinite generations.

In [8]:
EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN


def formatting_prompts_func(examples):
    inputs = examples["Question"]
    cots = examples["Complex_CoT"]
    outputs = examples["Response"]
    texts = []
    for input, cot, output in zip(inputs, cots, outputs):
        text = train_prompt_style.format(input, cot, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,
    }

In [9]:
from datasets import load_dataset
dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", 'zh', split = "train[0:5]", trust_remote_code=True)
print(dataset.column_names)

README.md:   0%|          | 0.00/1.65k [00:00<?, ?B/s]

medical_o1_sft_Chinese.json:   0%|          | 0.00/64.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/24772 [00:00<?, ? examples/s]

['Question', 'Complex_CoT', 'Response']


For `Ollama` and `llama.cpp` to function like a custom `ChatGPT` Chatbot, we must only have 2 columns - an `instruction` and an `output` column. We need to transform the dataset into proper structure.

In [10]:
dataset = dataset.map(formatting_prompts_func, batched = True)
dataset["text"][0]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

'Below is an instruction that describes a task, paired with an input that provides further context.\nWrite a response that appropriately completes the request.\nBefore answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.\n\n### Instruction:\nYou are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.\nPlease answer the following medical question.\n\n### Question:\n根据描述，一个1岁的孩子在夏季头皮出现多处小结节，长期不愈合，且现在疮大如梅，溃破流脓，口不收敛，头皮下有空洞，患处皮肤增厚。这种病症在中医中诊断为什么病？\n\n### Response:\n<think>\n这个小孩子在夏天头皮上长了些小结节，一直都没好，后来变成了脓包，流了好多脓。想想夏天那么热，可能和湿热有关。才一岁的小孩，免疫力本来就不强，夏天的湿热没准就侵袭了身体。\n\n用中医的角度来看，出现小结节、再加上长期不愈合，这些症状让我想到了头疮。小孩子最容易得这些皮肤病，主要因为湿热在体表郁结。\n\n但再看看，头皮下还有空洞，这可能不止是简单的头疮。看起来病情挺严重的，也许是脓肿没治好。这样的情况中医中有时候叫做禿疮或者湿疮，也可能是另一种情况。\n\n等一下，头皮上的空洞和皮肤增厚更像是疾病已经深入到头皮下，这是不是说明有可能是流注或瘰疬？这些名字常描述头部或颈部的严重感染，特别是有化脓不愈合，又形成通道或空洞的情况。\n\n仔细想想，我怎么感觉这些症状更贴近瘰疬的表现？尤其考虑到孩子的年纪和夏天发生的季节性因素，湿热可能是主因，但可能也有火毒或者痰湿造成的滞留。\n\n回到基本

## Train the model
Now let's use Huggingface TRL's `SFTTrainer`.

In [11]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)
FastLanguageModel.for_inference(model)

Unsloth 2025.2.15 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 1536, padding_idx=151654)
        (layers): ModuleList(
          (0): Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=1536, out_features=1536, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1536, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=1536, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(
       

In [12]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        # num_train_epochs = 1, # For longer training runs!
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Converting train dataset to ChatML (num_proc=2):   0%|          | 0/5 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/5 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/5 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/5 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 5 | Num Epochs = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 18,464,768


## Inference after fine-tuning

Let's inference with same question again and see the difference.

In [14]:
print(question)

根据描述，一个1岁的孩子在夏季头皮出现多处小结节，长期不愈合，且现在疮大如梅，溃破流脓，口不收敛，头皮下有空洞，患处皮肤增厚。这种病症在中医中诊断为什么病？


In [16]:
FastLanguageModel.for_inference(model)  # Unsloth has 2x faster inference!
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])


<think>
好，我来仔细分析一下这个问题。首先，孩子1岁，说明可能刚出生，所以他的皮肤可能比较敏感，容易出现各种问题。现在的情况是头皮出现小结节，长期不愈合，说明这些结节可能比较难处理，已经比较严重。现在这个疮大如梅，溃破流脓，口不收敛，头皮下有空洞，皮肤增厚。这些症状提示了严重的皮肤感染。

首先，结合孩子的年龄，可能属于新生儿期，因此皮肤结构可能比较脆弱，容易出现各种问题。小结节是常见的皮肤科病，但长期不愈合，说明可能需要进一步的诊断。

接下来，考虑中医诊断，中医通常会有 palp、指腹、推拿、脉考等方法来诊断。结合以上症状，首先可以考虑推拿法，观察是否有红肿、疼痛，如果有，可能需要进一步的检查。

然后，考虑是否需要进行皮肤科的检查，比如皮肤穿刺，检查是否有真菌感染、细菌感染或其他感染。如果真菌感染，可能需要抗生素治疗，但这里的问题是严重，可能需要更深入的诊断。

此外，考虑是否有感染性发热，比如热退、湿疹等，但这里症状是脓性感染，可能不是发热引起的。

综上所述，综合考虑中医和现代医学的诊断方法，这个病可能需要进一步的皮肤科诊断，比如细菌感染、真菌感染或感染热，同时可能需要考虑是否有感染性发热等，但根据症状描述，最可能的诊断是感染性发热，因为脓性感染可能不太常见于新生儿期，而发热则可能是因为细菌感染。

因此，综合考虑，这个症状可能属于发热，或者可能是感染热，结合小结节和脓性感染，更可能属于感染热，因为发热通常伴随红肿、疼痛，而脓性感染可能伴随结节和空洞，结合两者的症状，可能更倾向于感染热。
</think>

根据描述，这个症状可能属于感染热，因为有红肿、疼痛，结节和空洞，以及脓性流脓。结合中医和现代医学的诊断，可能需要进一步的皮肤科检查，如穿刺检查，以确定真菌感染、细菌感染或其他感染。<｜end▁of▁sentence｜>

<think>
好的，我现在需要分析一个1岁的孩子的情况，判断其在中医中的诊断。首先，孩子在夏季头皮出现多处小结节，长期不愈合，现在疮大如梅，溃破流脓，口不收敛，头皮下有空洞，患处皮肤增厚。这是一幅比较复杂的临床描述，我需要逐一分析。

首先，小结节是头皮常见的结构，通常与头皮的某些组织有关。小结节可能与头皮中的神经结构有关，比如神经根或神经节。长期不愈合通常提示神经结构有问题，可能与神经损伤或神经炎有关。

接下来，现在疮

## Upload Model to HuggingFace

Now, let's save our finetuned model and upload it to HuggingFace.

### Save the fine-tuned model to GGUF format

Choose the llama.cpp's GGUF format we prefer by setting the corresponding `if` to `True`.

In [ ]:
from google.colab import userdata

HUGGINGFACE_TOKEN = userdata.get('HUGGINGFACE_TOKEN')

In [ ]:
# Save to 8bit Q8_0
if True: model.save_pretrained_gguf("model", tokenizer,)

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model_f16", tokenizer, quantization_method = "f16")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### Your chat template has a BOS token. We shall remove it temporarily.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 6.0G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 34.71 out of 52.96 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 91%|█████████ | 29/32 [00:01<00:00, 18.35it/s]
We will save to Disk and not RAM now.
100%|██████████| 32/32 [00:02<00:00, 12.46it/s]


Unsloth: Saving tokenizer... Done.
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q8_0'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: CMAKE detected. Finalizing some steps for installation.
Unsloth: [1] Converting model at model into q8_0 GGUF format.
The output location will be /content/model/unsloth.Q8_0.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {64}
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model part 'model-00001-of-00004.safetensors

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### We removed it in GGUF's chat template for you.


Unsloth: Conversion completed! Output location: /content/model/unsloth.Q8_0.gguf


### Push the model to HuggingFace

Create a model type repository for your model if you haven't done so.

In [ ]:
from huggingface_hub import create_repo
create_repo("wyang14/medical-model", token=HUGGINGFACE_TOKEN, exist_ok=True)

RepoUrl('https://huggingface.co/wyang14/medical-model', endpoint='https://huggingface.co', repo_type='model', repo_id='wyang14/medical-model')

In [ ]:
model.push_to_hub_gguf("wyang14/medical-model", tokenizer, token = HUGGINGFACE_TOKEN)

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### Your chat template has a BOS token. We shall remove it temporarily.


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 33.04 out of 52.96 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 32/32 [00:02<00:00, 13.40it/s]


Unsloth: Saving tokenizer... Done.
Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q8_0'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: [1] Converting model at wyang14/medical-model into q8_0 GGUF format.
The output location will be /content/wyang14/medical-model/unsloth.Q8_0.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: medical-model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {64}
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model part 'model-00001-of-0

  0%|          | 0/1 [00:00<?, ?it/s]

unsloth.Q8_0.gguf:   0%|          | 0.00/8.54G [00:00<?, ?B/s]

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### We removed it in GGUF's chat template for you.


Saved GGUF to https://huggingface.co/wyang14/medical-model


### Ollama run HuggingFace model

```bash
ollama run hf.co/{username}/{repository}:{quantization}
```

### Ollama inference

```bash
curl http://localhost:11434/api/chat -d '{ \
  "model": "", \
  "messages": [ \
    { "role": "user", "content": "一个患有急性阑尾炎的病人已经发病5天，腹痛稍有减轻但仍然发热，在体检时发现右下腹有压痛的包块，此时应如何处理？" } \
  }'
```